In [1]:
import os
import mne
import json
import pickle
import ndx_events
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO

In [92]:
with open('../settings.json', "r") as f:
    settings = json.load(f)
    
epoch_folder = settings['epochs_folder']
plot_folder = settings['plots_folder']
nwb_folder = settings['nwb_files_folder']

Let's create some handy converting functions

In [93]:
def sample_to_frame(tll_onset_secs, fps, offset):
    """
    Function that calculates the time-point of the video (in frames) given
     the sample number in the EEG.
    """
    secs_video = tll_onset_secs - offset
    
    return secs_video * fps

In [94]:
def frame_to_sample(frame_number, fps, offset):
    """
    Function that calculates the time-point of the video (in frames) given
     the sample number in the EEG.
    """
    tp_secs_video = frame_number / fps  # time-point on video in seconds
    secs_eeg = tp_secs_video + offset
    
    return secs_eeg

### Now, let's test and validate the converting process

More sophisticated approach; get all LED onset frame numbers and calculate the seconds/sample num of the EEG. If they are close enough, the method works.

First, let's get all EEG TTL onset timepoints

In [95]:
test_subject = "78233"  # is in batch 1 (see metadata excel sheets)

In [96]:
eeg_ttl_onsets_secs = []  # container (to make it accessible through notebook)
s_freq = 0  # placeholder

for file in os.listdir(nwb_folder):
    if test_subject in file and file.endswith(".nwb"):
        with NWBHDF5IO(f'{nwb_folder}/{file}', "r") as io:  # open it
            nwb = io.read()
            eeg_ttl_onsets_secs = list(nwb.acquisition["TTL_1"].timestamps)
            s_freq = nwb.acquisition['raw_EEG'].rate

Now that we have the TTL onset timepoints from the EEG of the test_subject, let's load the data that holds the LED ON timepoints from the accompanying video file.

In [97]:
folder_path = r"C:\Users\Olle de Jong\Documents\MSc Biology\rp2\rp2_data\resting_state\output\videos"
pickle_path = rf"{folder_path}\pickle"

led_states = 0
with open(f'{pickle_path}/led_states_batch1.pickle', "rb") as f:
    led_states = pickle.load(f)

movie_name = list(led_states.keys())[0]
led_states_data = list(led_states.values())[0]

We only need the frame numbers where the LED switches ON, not all frames where the LED is ON.

In [98]:
led_turns_on_indexes = np.where(np.logical_and(np.diff(led_states_data), led_states_data[1:]))[0] + 1  # get all False to True changes in the array

Using the frame number where the LED turns ON for the first time, we can calculate the needed offset.
We calculate this by subtracting the time elapsed between start of **video** and the first LED onset from the time elapsed between start of **EEG** recording and the first TTL onset.

In [106]:
video_fps = 30
first_ttl_onset = eeg_ttl_onsets_secs[0]
last_ttl_onset = eeg_ttl_onsets_secs[-1]
first_LED_onset = led_turns_on_indexes[0]
last_LED_onset = led_turns_on_indexes[-1]

offset = first_ttl_onset - first_LED_onset / video_fps  # TTL onset is in seconds, so we need to transform the LED onset (in frames) to seconds as well (divide by frames per second)
offset_end = last_ttl_onset - last_LED_onset / video_fps
offset_average = (offset + offset_end) / 2

In [107]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=30, offset=offset)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 13.0716, calculated: 13.0716
Onset 1. Actual EEG TTL onset: 14.0727, calculated: 14.0716
Onset 2. Actual EEG TTL onset: 15.0748, calculated: 15.0716
Onset 3. Actual EEG TTL onset: 25.2968, calculated: 25.2716
Onset 4. Actual EEG TTL onset: 26.298, calculated: 26.2716
Onset 5. Actual EEG TTL onset: 27.2992, calculated: 27.2716
Onset 6. Actual EEG TTL onset: 19744.0058, calculated: 19728.938266666664
Onset 7. Actual EEG TTL onset: 19745.0079, calculated: 19729.938266666664
Onset 8. Actual EEG TTL onset: 19746.01, calculated: 19730.938266666664


Now let's try the other way around

In [108]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=30, offset=offset)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: 120.0
Onset 1. Actual frame LED onset: 150, calculated: 150.033
Onset 2. Actual frame LED onset: 180, calculated: 180.096
Onset 3. Actual frame LED onset: 486, calculated: 486.75600000000003
Onset 4. Actual frame LED onset: 516, calculated: 516.7919999999999
Onset 5. Actual frame LED onset: 546, calculated: 546.828
Onset 6. Actual frame LED onset: 591596, calculated: 592048.026
Onset 7. Actual frame LED onset: 591626, calculated: 592078.089
Onset 8. Actual frame LED onset: 591656, calculated: 592108.152


And now let's try the same but with averaged offset

In [ ]:
# TODO continue here.. Calculation of average offset does not seem to be right. We might need the last frame number and max duration of EEG for this to calculate it backwards.
last_ttl_onset = eeg_ttl_onsets_secs[-1]
last_LED_onset = led_turns_on_indexes[-1]

offset_end = last_ttl_onset - last_LED_onset / video_fps
offset_average = (offset + offset_end) / 2

In [109]:

for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=30, offset=offset_average)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 13.0716, calculated: 20.607466666666717
Onset 1. Actual EEG TTL onset: 14.0727, calculated: 21.607466666666717
Onset 2. Actual EEG TTL onset: 15.0748, calculated: 22.607466666666717
Onset 3. Actual EEG TTL onset: 25.2968, calculated: 32.80746666666671
Onset 4. Actual EEG TTL onset: 26.298, calculated: 33.80746666666671
Onset 5. Actual EEG TTL onset: 27.2992, calculated: 34.80746666666671
Onset 6. Actual EEG TTL onset: 19744.0058, calculated: 19736.474133333333
Onset 7. Actual EEG TTL onset: 19745.0079, calculated: 19737.474133333333
Onset 8. Actual EEG TTL onset: 19746.01, calculated: 19738.474133333333


In [110]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=30, offset=offset_average)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: -106.0760000000015
Onset 1. Actual frame LED onset: 150, calculated: -76.04300000000153
Onset 2. Actual frame LED onset: 180, calculated: -45.98000000000152
Onset 3. Actual frame LED onset: 486, calculated: 260.67999999999853
Onset 4. Actual frame LED onset: 516, calculated: 290.7159999999984
Onset 5. Actual frame LED onset: 546, calculated: 320.7519999999985
Onset 6. Actual frame LED onset: 591596, calculated: 591821.95
Onset 7. Actual frame LED onset: 591626, calculated: 591852.0129999999
Onset 8. Actual frame LED onset: 591656, calculated: 591882.0759999999


### Plotting the events (before and after aligning)

#### Before

In [ ]:
positions = np.array([2, 4, 6])[:,np.newaxis] 
offsets = [2,4,6] 

plt.eventplot(positions, lineoffsets=offsets) 
plt.show() 

#### After

Let's load a single filtered epoch file

In [7]:
for file in os.listdir(epoch_folder):
    if test_subject in file and "filtered" in file and file.endswith(".fif"):
        epochs = mne.read_epochs(os.path.join(epoch_folder, file), preload=True)
        print(epochs)

Reading C:\Users\Olle de Jong\Documents\MSc Biology\rp2\rp2_data\resting_state\output\epochs\filtered_epochs_resting_state_79593-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.53 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
1108 matching events found
No baseline correction applied
0 projection items activated
<EpochsFIF |  1108 events (all good), 0 – 4.99853 s, baseline off, ~412.6 MB, data loaded, with metadata,
 '1': 1108>
